<a href="https://colab.research.google.com/github/Saputoa21/Machine-Translation/blob/main/generate_translation_MarianMT_BacisMT2025S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Produce translations from a pre-trained model



In [1]:
#install transformers
!pip install transformers[sentencepiece] #==4.51.3

In [2]:
#import models and tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-de", device_map={"":0})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [3]:
#upload source file and read
import codecs
file_name = "Vienna_Environmental.en-de.test.en.txt"
mt_output = open('Vienna_Environmental.en-de.test.marian.de', 'w', encoding='utf-8')
with codecs.open(file_name, 'r', 'utf-8') as src:
  for line in src:
    line = line.strip()
    encoded = tokenizer(line, return_tensors="pt", padding=True).to("cuda") #tokenize and pad (add zeros)
    generated_tokens = model.generate(**encoded) #generate tokens
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True) #de tokenize and remove special tokens
    print(translation[0], file=mt_output)

mt_output.close()

# TODO


1. Translate test set with beam of size 6 [decoding](https://huggingface.co/blog/how-to-generate)
2. Translate Vienna test set with greedy and beam

In [4]:
#upload source file and read
import codecs
file_name = "Vienna_Environmental.en-de.test.en.txt"
beam_output = open("Vienna_Environmental.en-de.test.marian.de.beam", 'w', encoding='utf-8')
greedy_output = open("Vienna_Environmental.en-de.test.marian.de.greedy", 'w', encoding='utf-8')

with codecs.open(file_name, 'r', 'utf-8') as src:
    for line in src:
        line = line.strip()
        encoded = tokenizer(line, return_tensors="pt", padding=True).to("cuda") #tokenize and pad (add zeros)

        # beam decoding
        generated_beam = model.generate(**encoded, num_beams=6, early_stopping=True)  #generate tokens and set num_beams to 6
        translation_beam = tokenizer.batch_decode(generated_beam, skip_special_tokens=True) #de tokenize and remove special tokens
        print(translation_beam[0], file=beam_output)

        # greedy decoding
        generated_greedy = model.generate(**encoded)
        translation_greedy = tokenizer.batch_decode(generated_greedy, skip_special_tokens=True)
        print(translation_greedy[0], file=greedy_output)

beam_output.close()
greedy_output.close()

# Hugging face Tokenizers

In [5]:
#tokenizers
output = tokenizer("Hello, y'all! How are you 😁 ?")
print(output)

output = tokenizer.tokenize("Hello, y'all! How are you 😁 ?")
print(output)

{'input_ids': [16816, 2, 5582, 22, 2819, 68, 650, 48, 41, 17, 51670, 4377, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['▁Hello', ',', '▁y', "'", 'all', '!', '▁How', '▁are', '▁you', '▁', '😁', '▁?']


In [6]:
#tokenizers batches

sequences = ["Beam search keeps track of several generated sequences (beams) at each time step.",
             "Greedy search is the default decoding strategy."]

output = tokenizer(sequences)

# Will pad the sequences up to the maximum sequence length
print('longest')
output = tokenizer(sequences, padding="longest")
print(output)
print('\n')

print('max lenght')
# Will pad the sequences up to the model max length
output = tokenizer(sequences, padding="max_length")
print(output)
print('\n')

print('max 8')
# Will pad the sequences up to the specified max length
output = tokenizer(sequences, padding="max_length", max_length=8)
print(output)
print('\n')

# Will truncate the sequences that are longer than the model max length
print('truncate')
output = tokenizer(sequences, truncation=True)
print(output)
print('\n')

# Will truncate the sequences that are longer than the specified max length
print('truncate max 8')
output = tokenizer(sequences, max_length=8, truncation=True)
print(output)
print('\n')

print('ids and special tokens')
output = tokenizer(sequences[0])
print(output["input_ids"])
print('\n')

tokens = tokenizer.tokenize(sequences[0])
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)
print(tokenizer.decode(output["input_ids"]))
print(tokenizer.decode(ids))
print(tokenizer.special_tokens_map)

longest
{'input_ids': [[312, 982, 1582, 14165, 4470, 7, 1147, 8742, 32113, 20, 42195, 6, 27, 67, 506, 160, 2366, 3, 0], [6974, 108, 132, 1582, 19, 4, 9740, 230, 46703, 2781, 3, 0, 58100, 58100, 58100, 58100, 58100, 58100, 58100]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]]}


max lenght
{'input_ids': [[312, 982, 1582, 14165, 4470, 7, 1147, 8742, 32113, 20, 42195, 6, 27, 67, 506, 160, 2366, 3, 0, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 581